# NMX Reduction Workflow

> NMX does not expect users to use python interface directly.<br>
This documentation is mostly for instrument data scientists or instrument scientists.<br>

## TL;DR

In [ ]:
from ess.nmx.executables import reduction
from ess.nmx.data import get_small_nmx_nexus
from ess.nmx.configurations import (
    ReductionConfig,
    OutputConfig,
    InputConfig,
    WorkflowConfig,
    TimeBinCoordinate,
)

# Build Configuration
config = ReductionConfig(
    inputs=InputConfig(
        input_file=[get_small_nmx_nexus().as_posix()],
        detector_ids=[0, 1, 2],
    ),
    output=OutputConfig(
        output_file="scipp_output.hdf", skip_file_output=False, overwrite=True
    ),
    workflow=WorkflowConfig(
        time_bin_coordinate=TimeBinCoordinate.time_of_flight,
        nbins=10,
        tof_simulation_num_neutrons=1_000_000,
        tof_simulation_min_wavelength=1.8,
        tof_simulation_max_wavelength=3.6,
        tof_simulation_seed=42,
    ),
)

# Run Reduction
reduction(config=config, display=display)

## Configuration

`essnmx` provides a command line data reduction tool.<br>
The `essnmx-reduce` interface will reduce `nexus` file <br>
and save the results into `NXlauetof`(not exactly but very close) format for `dials`.<br>

Argument options could be exhaustive therefore we wrapped them into a nested pydantic model.<br>
Here is a python API you can use to build the configuration and turn it into command line arguments.

**The configuration object is a pydantic model, and it thus enforces strict checks on the types of the arguments.**

In [ ]:
from ess.nmx.configurations import (
    ReductionConfig,
    OutputConfig,
    InputConfig,
    WorkflowConfig,
    TimeBinCoordinate,
    to_command_arguments,
)

config = ReductionConfig(
    inputs=InputConfig(
        input_file=["PATH_TO_THE_NEXUS_FILE.hdf"],
        detector_ids=[0, 1, 2],  # Detector index to be reduced in alphabetical order.
    ),
    output=OutputConfig(output_file="scipp_output.hdf", skip_file_output=True),
    workflow=WorkflowConfig(
        time_bin_coordinate=TimeBinCoordinate.time_of_flight,
        nbins=10,
        tof_simulation_num_neutrons=1_000_000,
        tof_simulation_min_wavelength=1.8,
        tof_simulation_max_wavelength=3.6,
        tof_simulation_seed=42,
    ),
)

display(config)
print(to_command_arguments(config=config, one_line=True))

## Reduce Nexus File(s)

`OutputConfig` has an option called `skip_file_output` if you want to reduce the file and use it only on the memory.<br>
Then you can use `save_results` function to explicitly save the results.

In [ ]:
from ess.nmx.executables import reduction
from ess.nmx.data import get_small_nmx_nexus

config = ReductionConfig(
    inputs=InputConfig(input_file=[get_small_nmx_nexus().as_posix()]),
    output=OutputConfig(skip_file_output=True),
)
results = reduction(config=config, display=display)
results

In [ ]:
from ess.nmx.executables import save_results

output_config = OutputConfig(output_file="scipp_output.hdf", overwrite=True)
save_results(results=results, output_config=output_config)